<a href="https://colab.research.google.com/github/rajjat0602/GCP-PYTHON/blob/master/Apache_beam_batch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!{'pip install --quiet apache_beam'}

     |████████████████████████████████| 9.8 MB 2.2 MB/s 
     |████████████████████████████████| 249 kB 69.3 MB/s 
     |████████████████████████████████| 151 kB 64.9 MB/s 
     |████████████████████████████████| 62 kB 839 kB/s 
     |████████████████████████████████| 2.3 MB 49.0 MB/s 
     |████████████████████████████████| 829 kB 74.9 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
multiprocess 0.70.12.2 requires dill>=0.3.4, but you have dill 0.3.1.1 which is incompatible.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.26.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [3]:
!python --version

Python 3.7.12


In [4]:
!{'mkdir -p data'}

In [5]:
ls

data/  sample_data/


In [8]:
from google.colab import files
uploaded= files.upload()

Saving dept_data.txt to dept_data.txt


In [9]:
ls

data/  dept_data.txt  Retail_Data_Transactions.csv  sample_data/


In [12]:
import apache_beam as beam

def spiltRow(element):
  return element.split(',')
def filtering(record):
  return record[3] == 'Accounts'

p1=beam.Pipeline()

attendance_count = (
    
   p1
    |beam.io.ReadFromText('dept_data.txt')
    |beam.Map(spiltRow)
    |beam.Filter(filtering)  
    |beam.Map(lambda record: (record[1], 1))
    |beam.CombinePerKey(sum)
    |beam.io.WriteToText('data/output_new_final')
  
)

p1.run()

!{('head -n 20 data/output_new_final-00000-of-00001')}


('Marco', 31)
('Rebekah', 31)
('Itoe', 31)
('Edouard', 31)
('Kyle', 62)
('Kumiko', 31)
('Gaston', 31)
('Ayumi', 30)


In [13]:
# Branching P collection
import apache_beam as beam

def SplitRow(element):
    return element.split(',')

p = beam.Pipeline()


input_collection = ( 
                      p 
                      | "Read from text file" >> beam.io.ReadFromText('dept_data.txt')
                      | "Split rows" >> beam.Map(SplitRow)
                   )

accounts_count = (
                      input_collection
                      | 'Get all Accounts dept persons' >> beam.Filter(lambda record: record[3] == 'Accounts')
                      | 'Pair each accounts employee with 1' >> beam.Map(lambda record: ("Accounts, " +record[1], 1))
                      | 'Group and sum1' >> beam.CombinePerKey(sum)
                 )

hr_count = (
                input_collection
                | 'Get all HR dept persons' >> beam.Filter(lambda record: record[3] == 'HR')
                | 'Pair each hr employee with 1' >> beam.Map(lambda record: ("HR, " +record[1], 1))
                | 'Group and sum' >> beam.CombinePerKey(sum)
           )

output =(
         (accounts_count,hr_count)
    | beam.Flatten()
    | beam.io.WriteToText('data/both')
)

p.run()
!{('head -n 20 data/both-00000-of-00001')}

('HR, Beryl', 62)
('HR, Olga', 31)
('HR, Leslie', 31)
('HR, Mindy', 31)
('HR, Vicky', 31)
('HR, Richard', 31)
('HR, Kirk', 31)
('HR, Kaori', 31)
('HR, Oscar', 31)
('Accounts, Marco', 31)
('Accounts, Rebekah', 31)
('Accounts, Itoe', 31)
('Accounts, Edouard', 31)
('Accounts, Kyle', 62)
('Accounts, Kumiko', 31)
('Accounts, Gaston', 31)
('Accounts, Ayumi', 30)


In [14]:
#Doing same with ParDo
import apache_beam as beam

class SplitRow(beam.DoFn):
  
  def process(self, element):
    return  [element.split(',')]
  

class FilterAccountsEmployee(beam.DoFn):
  def process(self, element):
    if element[3] == 'Accounts':
      return [element]  
    
class PairEmployees(beam.DoFn):
  def process(self, element):
    return [(element[3]+","+element[1], 1)]    
  
class Counting(beam.DoFn):
  
  def process(self, element):
    # return type -> list
    (key, values) = element       
    return [(key, sum(values))]
     

p1 = beam.Pipeline()

attendance_count = (
    
   p1
    |beam.io.ReadFromText('dept_data.txt')
    |beam.ParDo(SplitRow())
    |beam.ParDo(FilterAccountsEmployee())
    |beam.ParDo(PairEmployees())
    | 'Group ' >> beam.GroupByKey()
    | 'Sum using ParDo' >> beam.ParDo(Counting())  
    
    |beam.io.WriteToText('data/output_new_final')
  
)

p1.run()

!{('head -n 20 data/output_new_final-00000-of-00001')}

('Accounts,Marco', 31)
('Accounts,Rebekah', 31)
('Accounts,Itoe', 31)
('Accounts,Edouard', 31)
('Accounts,Kyle', 62)
('Accounts,Kumiko', 31)
('Accounts,Gaston', 31)
('Accounts,Ayumi', 30)


In [15]:
#Advanced combiner
import apache_beam as beam

p = beam.Pipeline()

class AverageFn(beam.CombineFn): 
  def create_accumulator(self):
     return (0.0, 0)

  def add_input(self, sum_count, input):
    (sum, count) = sum_count
    return sum + input, count + 1

  def merge_accumulators(self, accumulators):
    
    ind_sums, ind_counts = zip(*accumulators) 
    return sum(ind_sums), sum(ind_counts)  

  def extract_output(self, sum_count):    
    
    (sum, count) = sum_count
    return sum / count if count else float('NaN')
  

small_sum = (
           p 
            | beam.Create([15,5,7,7,9,23,13,5])
            | "Combine Globally" >> beam.CombineGlobally(AverageFn()) 
            | 'Write results' >> beam.io.WriteToText('data/combine')
          )
p.run()
!{'head -n 20 data/combine-00000-of-00001'}

10.5
